In [127]:
import os

data_folder = '/kaggle/input/spine-x-rays-vertebrae-detection/BUU-LSPINE_400'

for root, dirs, files in os.walk(data_folder):
    level = root.replace(data_folder, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print('{}{}/'.format(indent, os.path.basename(root)))
    subindent = ' ' * 4 * (level + 1)
    for f in files:
        print('{}{}'.format(subindent, f))


BUU-LSPINE_400/
    metadata.csv
    yolov8_config.yaml
    BUU-LSPINE_400_report.xlsx
    AP/
        0363-F-062Y0.jpg
        0293-M-068Y0.csv
        0256-M-056Y0.csv
        0333-F-070Y0.jpg
        0022-M-006Y0.csv
        0090-F-016Y0.csv
        0460-F-019Y0.csv
        0070-F-027Y0.jpg
        0328-M-083Y0.jpg
        0107-F-033Y0.jpg
        0324-F-061Y0.csv
        0429-M-083Y0.csv
        0416-F-022Y0.jpg
        0198-F-063Y0.csv
        0480-M-052Y0.jpg
        0273-M-056Y0.csv
        0105-F-009Y0.jpg
        0245-F-086Y0.csv
        0148-M-058Y0.csv
        0158-F-072Y0.csv
        0458-M-020Y0.csv
        0040-M-023Y0.jpg
        0325-M-039Y0.csv
        0319-F-040Y0.jpg
        0327-F-079Y0.csv
        0412-F-023Y0.jpg
        0379-M-040Y0.jpg
        0385-F-071Y0.csv
        0258-F-058Y0.csv
        0151-M-079Y0.jpg
        0391-M-046Y0.csv
        0101-F-022Y0.jpg
        0418-F-025Y0.jpg
        0061-F-021Y0.csv
        0377-F-045Y0.csv
        0423-M-065Y0.csv
     

In [128]:
import os
import shutil

# Define paths
data_folder = '/kaggle/input/spine-x-rays-vertebrae-detection/BUU-LSPINE_400'

# Define writable paths in /kaggle/working
working_folder = '/kaggle/working/BUU-LSPINE_400'
train_images_folder = os.path.join(working_folder, 'train', 'images')
val_images_folder = os.path.join(working_folder, 'val', 'images')
test_images_folder = os.path.join(working_folder, 'test', 'images')
ap_folder = os.path.join(working_folder, 'AP')
la_folder = os.path.join(working_folder, 'LA')

# Function to create directories if they don't exist
def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f'Created directory: {directory}')

# Create writable directories
create_dir(train_images_folder)
create_dir(val_images_folder)
create_dir(test_images_folder)
create_dir(ap_folder)
create_dir(la_folder)

# Function to copy images based on naming convention
def copy_images_by_naming_convention(folder_path, destination_ap, destination_la):
    # List images in folder
    images = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
    # Copy each image to appropriate destination folder
    for image in images:
        if image.endswith('0.jpg'):
            destination_folder = destination_ap
        elif image.endswith('1.jpg'):
            destination_folder = destination_la
        else:
            continue  # Skip if not AP or LA image

        src = os.path.join(folder_path, image)
        dst = os.path.join(destination_folder, image)
        shutil.copy(src, dst)
        print(f'Copied {image} to {destination_folder}')


# Copy train images
copy_images_by_naming_convention(train_images_folder, ap_folder, la_folder)

# Copy validation images
copy_images_by_naming_convention(val_images_folder, ap_folder, la_folder)

# Copy test images
copy_images_by_naming_convention(test_images_folder, ap_folder, la_folder)


In [135]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import random

# Function to load and display an image with corresponding points from CSV
def display_random_image_with_points(image_dir, csv_dir, view):
    # List all image files in the directory
    image_files = os.listdir(image_dir)
    random_image_file = random.choice(image_files)

    # Construct paths
    image_path = os.path.join(image_dir, random_image_file)
    csv_path = os.path.join(csv_dir, random_image_file.replace('.jpg', '.csv').replace('.jpeg', '.csv').replace('.png', '.csv'))

    # Print debug information
    print(f"Image Path: {image_path}")
    print(f"CSV Path: {csv_path}")

    # Check file existence
    if not os.path.exists(image_path):
        print(f"Image file does not exist: {image_path}")
        return
    if not os.path.exists(csv_path):
        print(f"CSV file does not exist: {csv_path}")
        return

    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to read image: {image_path}")
        return
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Read the CSV file
    points = pd.read_csv(csv_path, header=None)

    # Define labels for AP and LA views
    if 'AP' in view:
        labels = ['L1a, 1', 'L1a, 2', 'L1b, 1', 'L1b, 2',
                  'L2a, 1', 'L2a, 2', 'L2b, 1', 'L2b, 2',
                  'L3a, 1', 'L3a, 2', 'L3b, 1', 'L3b, 2',
                  'L4a, 1', 'L4a, 2', 'L4b, 1', 'L4b, 2',
                  'L5a, 1', 'L5a, 2', 'L5b, 1', 'L5b, 2']
    elif 'LA' in view:
        labels = ['L1a, 1', 'L1a, 2', 'L1b, 1', 'L1b, 2',
                  'L2a, 1', 'L2a, 2', 'L2b, 1', 'L2b, 2',
                  'L3a, 1', 'L3a, 2', 'L3b, 1', 'L3b, 2',
                  'L4a, 1', 'L4a, 2', 'L4b, 1', 'L4b, 2',
                  'L5a, 1', 'L5a, 2', 'L5b, 1', 'L5b, 2',
                  'S1a, 1', 'S1a, 2']

    # Plot the image
    plt.figure(figsize=(10, 10))
    plt.imshow(image)

    # Plot the points with annotations
    for index in range(0, len(points) - 1, 2):  # Iterate by pairs of edge lines
        # Check if enough rows remain to access the next pair of points
        if index + 1 < len(points):
            x1, y1, x2, y2, _ = points.iloc[index]  # First point
            x3, y3, x4, y4, _ = points.iloc[index + 1]  # Second point

            # Get unique labels for this pair
            label1 = labels[index // 2 * 4]
            label2 = labels[index // 2 * 4 + 1]
            label3 = labels[index // 2 * 4 + 2]
            label4 = labels[index // 2 * 4 + 3]

            # Plot points as small yellow circles
            plt.scatter([x1, x2], [y1, y2], color='yellow', s=15)

            # Display labels near the points
            plt.text(x1, y1, label1, color='yellow', fontsize=6, ha='right', va='top')
            plt.text(x2, y2, label2, color='yellow', fontsize=6, ha='left', va='top')

            # Connect the points with a line
            plt.plot([x1, x2], [y1, y2], color='green')

            # Plot points for the second pair
            plt.scatter([x3, x4], [y3, y4], color='yellow', s=15)

            # Display labels near the points for the second pair
            plt.text(x3, y3, label3, color='yellow', fontsize=6, ha='right', va='top')
            plt.text(x4, y4, label4, color='yellow', fontsize=6, ha='left', va='top')

            # Connect the points of the second pair with a line
            plt.plot([x3, x4], [y3, y4], color='green')

    # Add the first sacral vertebra edge line (points 21 and 22) for LA view
    if 'LA' in view and len(points) > 10:
        x1, y1, x2, y2, _ = points.iloc[10]  # S1a, 1 and S1a, 2

        # Get labels for sacral vertebra
        label1 = labels[20]
        label2 = labels[21]

        # Plot points as small yellow circles
        plt.scatter([x1, x2], [y1, y2], color='yellow', s=15)

        # Display labels near the points
        plt.text(x1, y1, label1, color='yellow', fontsize=6, ha='right', va='top')
        plt.text(x2, y2, label2, color='yellow', fontsize=6, ha='left', va='top')

        # Connect the points with a line
        plt.plot([x1, x2], [y1, y2], color='green')

    plt.title(f'Random {view} Image with Annotation Edge Lines')
    plt.axis('off')  # Hide axis for cleaner visualization
    plt.show()

# Example paths
ap_image_dir = '/kaggle/input/spine-x-rays-vertebrae-detection/BUU-LSPINE_400/AP'  # Directory containing AP view images
ap_csv_dir = '/kaggle/input/spine-x-rays-vertebrae-detection/BUU-LSPINE_400/AP'  # Directory containing corresponding CSV files

la_image_dir = '/kaggle/input/spine-x-rays-vertebrae-detection/BUU-LSPINE_400/LA'  # Directory containing LA view images
la_csv_dir = '/kaggle/input/spine-x-rays-vertebrae-detection/BUU-LSPINE_400/LA'  # Directory containing corresponding CSV files

# Display random image with points for AP and LA views
display_random_image_with_points(ap_image_dir, ap_csv_dir, 'AP')
display_random_image_with_points(la_image_dir, la_csv_dir, 'LA')


Image Path: /kaggle/input/spine-x-rays-vertebrae-detection/BUU-LSPINE_400/AP/0436-F-054Y0.csv
CSV Path: /kaggle/input/spine-x-rays-vertebrae-detection/BUU-LSPINE_400/AP/0436-F-054Y0.csv
Unable to read image: /kaggle/input/spine-x-rays-vertebrae-detection/BUU-LSPINE_400/AP/0436-F-054Y0.csv
Image Path: /kaggle/input/spine-x-rays-vertebrae-detection/BUU-LSPINE_400/LA/0201-F-077Y1.csv
CSV Path: /kaggle/input/spine-x-rays-vertebrae-detection/BUU-LSPINE_400/LA/0201-F-077Y1.csv
Unable to read image: /kaggle/input/spine-x-rays-vertebrae-detection/BUU-LSPINE_400/LA/0201-F-077Y1.csv


In [130]:
filename = "0020-M-030Y1.jpg"

# Split filename by hyphen to extract parts
parts = filename.split('-')

# Extract Patient_ID (parts[0])
patient_id = parts[0]

# Extract Gender (parts[1])
gender = parts[1]

# Extract Age (parts[2][:3] to get the first three characters)
age = parts[2][:3]  # Extracts "037"

# Extract View_Type (last character of parts[2])
view_type = 'AP' if parts[2][-1] == '0' else 'LA'

print(f"Patient_ID: {patient_id}")
print(f"Gender: {gender}")
print(f"Age: {age}")
print(f"View_Type: {view_type}")


Patient_ID: 0020
Gender: M
Age: 030
View_Type: LA


In [136]:
from sklearn.model_selection import train_test_split

import pandas as pd
import os
import shutil

# Function to update paths in metadata CSV
def update_metadata_paths(metadata_csv_path, data_folder):
    df = pd.read_csv(metadata_csv_path)

    updated_image_paths = []
    updated_csv_paths = []

    for index, row in df.iterrows():
        image_path = row['Image_Path']
        csv_path = row['CSV_Path']

        # Extract image name
        image_name = os.path.basename(image_path)
        csv_name = os.path.basename(csv_path)
        
        # Update paths
        updated_image_path = os.path.join(data_folder, row['View_Type'], image_name)
        updated_csv_path = os.path.join(data_folder, row['View_Type'], csv_name)

        updated_image_paths.append(updated_image_path)
        updated_csv_paths.append(updated_csv_path)

    df['Image_Path'] = updated_image_paths
    df['CSV_Path'] = updated_csv_paths

    # Save updated metadata CSV
    updated_metadata_path = os.path.join('/kaggle/working', os.path.basename(metadata_csv_path))
    df.to_csv(updated_metadata_path, index=False)
    print(f"Updated metadata saved to: {updated_metadata_path}")

# Define paths
data_folder = '/kaggle/input/spine-x-rays-vertebrae-detection/BUU-LSPINE_400'
metadata_csv_path = '/kaggle/input/spine-x-rays-vertebrae-detection/BUU-LSPINE_400/metadata.csv'
working_metadata_path = '/kaggle/working/metadata.csv'

# Copy metadata CSV to working directory
shutil.copy(metadata_csv_path, working_metadata_path)

# Update paths in metadata CSV
update_metadata_paths(working_metadata_path, data_folder)



# Split the dataset into train, validation, and test sets (80%, 10%, 10%)
train_df, val_test_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=42)

# Save train, validation, and test CSVs
train_df.to_csv(train_csv_path, index=False)
val_df.to_csv(val_csv_path, index=False)
test_df.to_csv(test_csv_path, index=False)

print(f"Train metadata saved to: {train_csv_path}")
print(f"Validation metadata saved to: {val_csv_path}")
print(f"Test metadata saved to: {test_csv_path}")


Updated metadata saved to: /kaggle/working/metadata.csv
Train metadata saved to: /kaggle/working/train_metadata.csv
Validation metadata saved to: /kaggle/working/val_metadata.csv
Test metadata saved to: /kaggle/working/test_metadata.csv


In [137]:
!pip install tensorflow pandas opencv-python

In [141]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

# Update paths to reflect the correct directory structure in Kaggle
train_metadata_path = '/kaggle/working/train_metadata.csv'
val_metadata_path = '/kaggle/working/val_metadata.csv'
test_metadata_path = '/kaggle/working/test_metadata.csv'
working_metadata_path = '/kaggle/working/metadata.csv'


# Function to load image and keypoints
def load_image_and_keypoints(image_path, csv_path, image_size=(640, 640)):
    # Check if image path exists
    if not tf.io.gfile.exists(image_path):
        raise FileNotFoundError(f"Image file not found: {image_path}")

    # Read image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, image_size)

    # Read keypoints
    keypoints = pd.read_csv(csv_path, header=None).values[:, :4].flatten()

    return image, keypoints

# Function to create dataset from metadata
def create_dataset(working_metadata_path, image_size=(640, 640), view='AP'):
    metadata = pd.read_csv(working_metadata_path)
    images = []
    keypoints = []

    for index, row in metadata.iterrows():
        if row['View_Type'] == view:
            image_path = row['Image_Path']
            csv_path = row['CSV_Path']
            try:
                image, kp = load_image_and_keypoints(image_path, csv_path, image_size=image_size)
                images.append(image)
                keypoints.append(kp)
            except Exception as e:
                print(f"Error loading {image_path}: {e}")

    images = np.array(images, dtype='float32') / 255.0
    keypoints = np.array(keypoints, dtype='float32')

    return images, keypoints

# Create datasets for AP and LA views separately
train_images_ap, train_keypoints_ap = create_dataset(train_metadata_path, view='AP')
train_images_la, train_keypoints_la = create_dataset(train_metadata_path, view='LA')

val_images_ap, val_keypoints_ap = create_dataset(val_metadata_path, view='AP')
val_images_la, val_keypoints_la = create_dataset(val_metadata_path, view='LA')

test_images_ap, test_keypoints_ap = create_dataset(test_metadata_path, view='AP')
test_images_la, test_keypoints_la = create_dataset(test_metadata_path, view='LA')


Error loading /content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400/AP/0314-M-015Y0.jpg: Image file not found: /content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400/AP/0314-M-015Y0.jpg
Error loading /content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400/AP/0431-F-066Y0.jpg: Image file not found: /content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400/AP/0431-F-066Y0.jpg
Error loading /content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400/AP/0441-M-044Y0.jpg: Image file not found: /content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400/AP/0441-M-044Y0.jpg
Error loading /content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400/AP/0491-F-018Y0.jpg: Image file not found: /content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400/AP/0491-F-018Y0.jpg
Error loading /content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400/AP/0102-F-013Y0.jpg: Image file not found: /content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400/AP/0102-F-013Y0.jpg
Error loading /content/dr

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# Define and compile ResNet50 model for AP view
def create_resnet_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(640, 640, 3))
    x = base_model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(40 if view == 'AP' else 44)(x)  # Adjust output size for AP and LA views
    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])  # Adjust loss and metrics as needed

    return model

# Create ResNet50 models for AP and LA views
resnet_ap = create_resnet_model(view='AP')
resnet_la = create_resnet_model(view='LA')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping



# Define callbacks for training
checkpoint_ap = ModelCheckpoint('/kaggle/working/resnet_ap.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
checkpoint_la = ModelCheckpoint('/kaggle/working/resnet_la.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
reduce_lr_ap = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
reduce_lr_la = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

# Train ResNet50 models
history_ap = resnet_ap.fit(train_images_ap, train_keypoints_ap, epochs=20, batch_size=4,
                           validation_data=(val_images_ap, val_keypoints_ap),
                           callbacks=[checkpoint_ap, reduce_lr_ap])

history_la = resnet_la.fit(train_images_la, train_keypoints_la, epochs=20, batch_size=4,
                           validation_data=(val_images_la, val_keypoints_la),
                           callbacks=[checkpoint_la, reduce_lr_la])

# Evaluate models on test set
ap_loss, ap_accuracy = resnet_ap.evaluate(test_images_ap, test_keypoints_ap)
la_loss, la_accuracy = resnet_la.evaluate(test_images_la, test_keypoints_la)

print(f"AP Model Accuracy: {ap_accuracy}")
print(f"LA Model Accuracy: {la_accuracy}")

# Define callbacks
callbacks = [
    ModelCheckpoint(filepath='best_model.h5', save_best_only=True, monitor='val_loss', mode='min'),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]

# Create and train ResNet50 model for AP view
ap_model = create_resnet_model(input_shape=(640, 640, 3))
ap_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
ap_history = ap_model.fit(ap_train_generator, epochs=20, validation_data=ap_val_generator, callbacks=callbacks)

# Create and train ResNet50 model for LA view
la_model = create_resnet_model(input_shape=(640, 640, 3))
la_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
la_history = la_model.fit(la_train_generator, epochs=20, validation_data=la_val_generator, callbacks=callbacks)


In [9]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


# Define callbacks
callbacks = [
    ModelCheckpoint(filepath='/kaggle/input/spine-x-rays-vertebrae-detection/BUU-LSPINE_400/best_model.h5', save_best_only=True, monitor='val_loss', mode='min'),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]




Functions to create MobileNetV2 model

In [10]:
#for AP
def create_mobilenetv2_model_AP(input_shape):
    base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
    x = Flatten()(base_model.output)
    x = Dense(1024, activation='relu')(x)
    x = Dense(40, activation='linear')(x)  # Adjust output shape according to your needs
    model = Model(inputs=base_model.input, outputs=x)
    return model

In [ ]:
# Create and train MobileNetV2 model for LA view
la_model = create_mobilenetv2_model_LA(input_shape=(640, 640, 3))
la_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
la_history = la_model.fit(la_train_generator, epochs=20, validation_data=la_val_generator, callbacks=callbacks)

In [ ]:
!pip install ultralytics


In [ ]:
import pandas as pd
import os

# Defining the class mapping
class_map = {
    'AP': {
        'APL1': 1, 'APL2': 2, 'APL3': 3, 'APL4': 4, 'APL5': 5
    },
    'LA': {
        'LAL1': 1, 'LAL2': 2, 'LAL3': 3, 'LAL4': 4, 'LAL5': 5,
        'S1a': 11
    }
}

# Function to convert coordinates to YOLO format
def convert_to_yolo_format(image_width, image_height, x1, y1, x2, y2):
    x_center = (x1 + x2) / 2.0 / image_width
    y_center = (y1 + y2) / 2.0 / image_height
    width = abs(x2 - x1) / image_width
    height = abs(y2 - y1) / image_height
    return x_center, y_center, width, height

# Function to generate class_id based on edge line and view
def get_class_id(edge_line, view):
    return class_map[view].get(edge_line, -1)  # Return -1 if not found

# Function to generate YOLO annotations from CSV
def generate_yolo_annotations(image_dir, csv_dir, output_dir, view):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for image_file in os.listdir(image_dir):
        if image_file.lower().endswith('.jpg') or image_file.lower().endswith('.jpeg') or image_file.lower().endswith('.png'):
            image_path = os.path.join(image_dir, image_file)
            csv_path = os.path.join(csv_dir, image_file.replace('.jpg', '.csv').replace('.jpeg', '.csv').replace('.png', '.csv'))

            if os.path.exists(csv_path):
                # Read the image to get its dimensions
                image = cv2.imread(image_path)
                image_height, image_width, _ = image.shape

                # Read the CSV file
                points = pd.read_csv(csv_path, header=None)

                # Prepare the output annotation file
                annotation_file = os.path.join(output_dir, image_file.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt'))

                with open(annotation_file, 'w') as f:
                    for index, row in points.iterrows():
                        x1, y1, x2, y2, _ = row  # Assuming CSV columns are x1, y1, x2, y2, class_id
                        class_id = get_class_id(f'L{index + 1}a', view)  # Assuming rows are labeled L1a, L2a, ..., S1a for LA view
                        if class_id != -1:  # Only write valid classes
                            x_center, y_center, width, height = convert_to_yolo_format(image_width, image_height, x1, y1, x2, y2)
                            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Example paths
ap_image_dir = '/content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400/AP'  # Directory containing AP view images
ap_csv_dir = '/content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400/AP'  # Directory containing corresponding CSV files
ap_output_dir = '/content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400/AP_YOLO_annotations'  # Output directory for YOLO annotations

la_image_dir = '/content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400/LA'  # Directory containing LA view images
la_csv_dir = '/content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400/LA'  # Directory containing corresponding CSV files
la_output_dir = '/content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400/LA_YOLO_annotations'  # Output directory for YOLO annotations

# Generate YOLO annotations for AP and LA views
generate_yolo_annotations(ap_image_dir, ap_csv_dir, ap_output_dir, 'AP')
generate_yolo_annotations(la_image_dir, la_csv_dir, la_output_dir, 'LA')


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
data_folder = '/content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400'
ap_folder = os.path.join(data_folder, 'AP')
la_folder = os.path.join(data_folder, 'LA')

# Function to gather images
def gather_images(data_folder):
    ap_images = [os.path.join(ap_folder, f) for f in os.listdir(ap_folder) if f.lower().endswith('.jpg')]
    la_images = [os.path.join(la_folder, f) for f in os.listdir(la_folder) if f.lower().endswith('.jpg')]
    return ap_images, la_images

# Function to split dataset
def split_dataset(images, val_ratio=0.1, test_ratio=0.1):
    train_images, temp_images = train_test_split(images, test_size=(val_ratio + test_ratio), random_state=42)
    val_images, test_images = train_test_split(temp_images, test_size=test_ratio / (val_ratio + test_ratio), random_state=42)
    return train_images, val_images, test_images

# Function to move files to respective folders
def move_files(file_list, dest_folder):
    os.makedirs(dest_folder, exist_ok=True)
    for file_name in file_list:
        shutil.move(file_name, dest_folder)
        print(f"Moved {file_name} to {dest_folder}")

# Gather AP and LA images
ap_images, la_images = gather_images(data_folder)

# Split AP and LA images into train, val, test
train_ap_images, val_ap_images, test_ap_images = split_dataset(ap_images)
train_la_images, val_la_images, test_la_images = split_dataset(la_images)

# Define folders
train_folder = os.path.join(data_folder, 'train', 'images')
val_folder = os.path.join(data_folder, 'val', 'images')
test_folder = os.path.join(data_folder, 'test', 'images')

# Move images to respective train, val, test folders
move_files(train_ap_images, train_folder)
move_files(val_ap_images, val_folder)
move_files(test_ap_images, test_folder)

move_files(train_la_images, train_folder)
move_files(val_la_images, val_folder)
move_files(test_la_images, test_folder)

# Display number of images in each set
print(f"Train AP images: {len(train_ap_images)}, Train LA images: {len(train_la_images)}")
print(f"Val AP images: {len(val_ap_images)}, Val LA images: {len(val_la_images)}")
print(f"Test AP images: {len(test_ap_images)}, Test LA images: {len(test_la_images)}")


In [ ]:
config_content = """
path: /content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400
train: train/images
val: val/images
test: test/images

nc: 1
names: ['vertebra']
"""

config_path = os.path.join(data_folder, 'yolov8_config.yaml')
with open(config_path, 'w') as f:
    f.write(config_content)


In [ ]:
from ultralytics import YOLO

# Define the path to your configuration file
data_folder = '/content/drive/My Drive/INTERNSHIP PROJECTS/BUU-LSPINE_400'
config_path = os.path.join(data_folder, 'yolov8_config.yaml')

# Loading YOLO model
model = YOLO('yolov8x.pt')

# Training the model with early stopping
model.train(data=config_path, epochs=100, imgsz=640, batch=16, patience=10)
